In [1]:
#export
"""
This module is for color formats, units and whatnot. This is exposed
automatically with::

   from k1lib.imports import *
   fmt.txt # exposed
"""
import k1lib, math; from k1lib import cli
from typing import Dict, Iterator, Tuple
__all__ = ["size", "sizeOf", "comp", "compRate", "time", "item", "txt"]

In [2]:
#export
metricPrefixes = {-8:"y",-7:"z",-6:"a",-5:"f",-4:"p",-3:"n",-2:"u",-1:"m",0:"",1:"k",2:"M",3:"G",4:"T",5:"P",6:"E",7:"Z",8:"Y"}
#metricPrefixes = ["", "k", "M", "G", "T", "P", "E", "Z", "Y"]
def _formatScale(x, units:Dict[int, str]):
    for i, unit in units.items():
        upperBound = 1000 * 1000**i
        if abs(x) < upperBound:
            return f"{round(1e3*x/upperBound, 2)} {unit}"
    return (f"{round(1e3*x/upperBound, 2)} {unit}").strip()

In [3]:
#export
sizes = {i: f"{p}B" for i, p in metricPrefixes.items() if i >= 0}; sizes[0] = "bytes"
def size(_bytes=0):
    """Formats disk size.
Example::

    # returns "50.0 bytes"
    fmt.size(50)
    # returns "12.0 MB"
    fmt.size(1.2e7)
"""
    return _formatScale(_bytes, sizes)

In [4]:
assert size(50) == "50.0 bytes"
assert size(1.2e7) == "12.0 MB"

In [5]:
#export
def sizeOf(l:Iterator[float]) -> Tuple[str, Iterator[float]]:
    """Figures out appropriate scale, scales back the Iterator, and return both.
Example::

    x = torch.abs(torch.randn(2)) * 1e4 + 1e5
    label, t = fmt.sizeOf(x) # label is "kB"
    (t | toTensor()).min() # min value should be close to 100
"""
    l = list(l)
    v = l | cli.toMax()
    v = math.log10(v) if v > 0 else -math.log10(-v)
    idx = math.floor(v/3)
    coef = 1.0/1000**idx
    return sizes[idx], l | cli.apply(lambda x: x * coef) | cli.deref()

In [6]:
import torch
x = torch.abs(torch.randn(2)) * 1e4 + 1e5
label, i = sizeOf(x); assert label == "kB"
assert 50 < (i | cli.toTensor()).min() < 150

In [7]:
#export
computations = {i: f"{p}FLOPs" for i, p in metricPrefixes.items() if i >= 0}
def comp(flop=0):
    """Formats computation amount.
Example::

    # returns "50.0 FLOPs"
    fmt.computation(50)
    # returns "50.0 MFLOPs"
    fmt.computation(5e7)
"""
    return _formatScale(flop, computations)

In [8]:
assert comp(50) == "50.0 FLOPs"
assert comp(5e7) == "50.0 MFLOPs"
assert comp(1e30) == "1000000.0 YFLOPs"

In [9]:
#export
computationRates = {i: f"{p}FLOPS" for i, p in metricPrefixes.items() if i >= 0}
def compRate(flops=0):
    """Formats computation rate.
Example::

    # returns "50.0 FLOPS"
    fmt.computationRate(50)
    # returns "50.0 MFLOPS"
    fmt.computationRate(5e7)
"""
    return _formatScale(flops, computationRates)

In [10]:
assert compRate(50) == "50.0 FLOPS"
assert compRate(5e7) == "50.0 MFLOPS"
assert compRate(1e30) == "1000000.0 YFLOPS"

In [11]:
#export
times = {i:f"{p}s" for i, p in metricPrefixes.items() if i <= 0}
def time(seconds=0):
    """Formats small times.
Example::

    fmt.time(50) # returns "50.0 s"
    fmt.time(4000) # returns "4000.0 s"
    fmt.time(0.02) # returns "20.0 ms"
    fmt.time(1e-5) # returns "10.0 us"
"""
    return _formatScale(seconds, times)

In [12]:
assert time(50) == "50.0 s"
assert time(4000) == "4000.0 s"
assert time(0.02) == "20.0 ms"
assert time(1e-5) == "10.0 us"
assert time(1e-10) == "100.0 ps"
assert time(1e-25) == "0.1 ys"

In [13]:
#export
items = {0: "", 1: "k", 2: "M", 3: "B", 4: "T"}
def item(n=0):
    """Formats generic item.
Example::

    # returns "50.0"
    fmt.item(50)
    # returns "500.0 k"
    fmt.item(5e5)
"""
    return _formatScale(n, items).strip()

In [14]:
assert item(50) == "50.0"
assert item(5e5) == "500.0 k"

In [15]:
#export
_esc = '\033['
_end = f'{_esc}0m'
class txt:
    """Text formatting.
Example::

    # will print out red text
    print(fmt.txt.red("some text"))"""
    @staticmethod
    def darkcyan(s:str):  return f"{_esc}36m{s}{_end}"
    @staticmethod
    def red(s:str):       return f"{_esc}91m{s}{_end}"
    @staticmethod
    def green(s:str):     return f"{_esc}92m{s}{_end}"
    @staticmethod
    def yellow(s:str):    return f"{_esc}93m{s}{_end}"
    @staticmethod
    def blue(s:str):      return f"{_esc}94m{s}{_end}"
    @staticmethod
    def purple(s:str):    return f"{_esc}95m{s}{_end}"
    @staticmethod
    def cyan(s:str):      return f"{_esc}96m{s}{_end}"
    @staticmethod
    def bold(s:str):      return f"{_esc}1m{s}{_end}"
    @staticmethod
    def grey(s:str):      return f"{_esc}38;2;150;150;150m{s}{_end}"
    @staticmethod
    def darkgrey(s:str):  return f"{_esc}38;2;100;100;100m{s}{_end}"
    @staticmethod
    def underline(s:str): return f"{_esc}4m{s}{_end}"
    @staticmethod
    def identity(s:str):  return f"{s}"

In [7]:
!../export.py fmt

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.11
Uninstalling k1lib-0.11:
  Successfully uninstalled k1lib-0.11
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/_context.py -> build/lib/k1lib
copying k1lib/selector.py -> build/li